In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
from siphon.simplewebservice.wyoming import WyomingUpperAir
import metpy.calc as mpcalc
import matplotlib.pyplot as plt
import metpy.calc as mpcalc
from metpy.cbook import get_test_data
from metpy.plots import Hodograph, SkewT
from metpy.units import units
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
#pink=0.81 https://github.com/Unidata/MetPy/issues/997
# https://pint.readthedocs.io/en/0.9/tutorial.html
%matplotlib inline

In [2]:
profile_units={'pressure': 'hPa', 'height': 'meter', 'temperature': 'degC', 'dewpoint': 'degC', 
          'direction': 'degrees', 'speed': 'knot', 'u_wind': 'knot', 'v_wind': 'knot', 
          'station': None, 'station_number': None,'time': None, 
          'latitude': 'degrees', 'longitude': 'degrees', 'elevation': 'meter'}

In [55]:
def caculate_man(df):
    df=df.dropna()#with out empty columns!!!
    p = df['pressure'].values * units(profile_units['pressure'])
    #p = df.index.values * units(profile_units['pressure']) 
    T = df['temperature'].values * units(profile_units['temperature'])
    Td = df['dewpoint'].values * units(profile_units['dewpoint'])
    u = df['u_wind'].values * units(profile_units['u_wind'])
    v = df['v_wind'].values * units(profile_units['v_wind'])
    wind_speed = df['speed'].values * units(profile_units['speed'])
    parcel_profile=mpcalc.parcel_profile(p, T[0], Td[0])
    cape0,cin0=mpcalc.cape_cin(p,T,Td,parcel_profile)
    cape,cin=mpcalc.surface_based_cape_cin(p, T, Td)
    cape1,cin1=mpcalc.most_unstable_cape_cin(p, T, Td)
    cape_man=cape.magnitude
    cape_man1=cape1.magnitude
    cin_man=cin.magnitude
    cin_man1=cin1.magnitude 
    lfcp,lfct=mpcalc.lfc(p, T, Td)
    lfcp_man=lfcp.magnitude
    lfct_man=lfct.magnitude 
    elp,elt=mpcalc.el(p,T,Td)
    lclp,lclt=mpcalc.lcl(p[0],T[0],Td[0])
    elp_man=elp.magnitude
    elt_man=elt.magnitude
    lclp_man=lclp.magnitude
    lclt_man=lclt.magnitude 
    bulk_u,bulk_v=mpcalc.bulk_shear(p,u,v)
    bulk_u_man=bulk_u.magnitude
    bulk_v_man=bulk_v.magnitude
    return cape0,cin0,cape_man,cin_man,cape_man1,cin_man1,lfcp_man,lfct_man,elp_man,elt_man,lclp_man,lclt_man,bulk_u_man,bulk_v_man

def caculate_cc(df):
    df=df.dropna()
    p = df['pressure'].values * units(profile_units['pressure'])
    print(p.shape)
    #p = df.index.values * units(profile_units['pressure']) 
    T = df['temperature'].values * units(profile_units['temperature'])
    print(T.shape)
    Td = df['dewpoint'].values * units(profile_units['dewpoint'])
    print(Td.shape)
    u = df['u_wind'].values * units(profile_units['u_wind'])
    v = df['v_wind'].values * units(profile_units['v_wind'])
    wind_speed = df['speed'].values * units(profile_units['speed'])
    parcel_profile=mpcalc.parcel_profile(p, T[0], Td[0])
    cape,cin=mpcalc.cape_cin(p,T,Td,parcel_profile)
    print(cape)
    print(cin)

In [4]:
def skewt_plot(df,stapng):
    df=df.dropna()
    p = df['pressure'].values * units(profile_units['pressure'])
    #p = df.index.values * units(profile_units['pressure'])
    T = df['temperature'].values * units(profile_units['temperature'])
    Td = df['dewpoint'].values * units(profile_units['dewpoint'])
    u = df['u_wind'].values * units(profile_units['u_wind'])
    v = df['v_wind'].values * units(profile_units['v_wind'])
    wind_speed = df['speed'].values * units(profile_units['speed'])
    cape,cin,cape1,cin1,lfc,el,lcl,bulk=caculate(df)
    df.set_index("pressure",inplace=True)
    fig = plt.figure(figsize=(9, 9))
    #add_metpy_logo(fig, 110, 100)
    skew = SkewT(fig, rotation=45)

    # Plot the data using normal plotting functions, in this case using
    # log scaling in Y, as dictated by the typical meteorological plot
    skew.plot(p, T, 'r')
    skew.plot(p, Td, 'g')
    skew.plot_barbs(p, u, v)
    skew.ax.set_ylim(1000, 100)
    skew.ax.set_xlim(-40, 60)
    #skew.ax.set_title(na+' '+str(date)[0:13]+' Skew-T', fontsize=20)
    skew.ax.set_title(' Skew-T', fontsize=20)
    skew.ax.set_xlabel(r'Temperature ($^{o}$c)',fontsize=16)
    skew.ax.set_ylabel('Pressure (hPa)',fontsize=16)
    skew.ax.tick_params(axis='both',width=2,labelsize=16)

    # Calculate LCL height and plot as black dot
    lcl_pressure, lcl_temperature = mpcalc.lcl(p[0], T[0], Td[0])
    skew.plot(lcl_pressure, lcl_temperature, 'ko', markerfacecolor='black')

    # Calculate full parcel profile and add to plot as black line

    prof = mpcalc.parcel_profile(p, T[0], Td[0]).to('degC')
    skew.plot(p, prof, 'k', linewidth=2)

    #print(T)
    #print(p)
    # Shade areas of CAPE and CIN
    skew.shade_cin(p, T, prof)
    skew.shade_cape(p, T, prof)

    # An example of a slanted line at constant T -- in this case the 0
    # isotherm
    skew.ax.axvline(0, color='c', linestyle='--',linewidth=2)

    # Add the relevant special lines
    skew.plot_dry_adiabats()
    skew.plot_moist_adiabats()
    skew.plot_mixing_lines()

    # Create a hodograph
    # Create an inset axes object that is 40% width and height of the
    # figure and put it in the upper right hand corner.
    #ax_hod = inset_axes(skew.ax, '20%', '20%', loc='upper left', bbox_to_anchor=(0.5, 1.05))
    ax_hod = inset_axes(skew.ax, '24%', '24%', loc='upper left')
    #ax_hod.set_yticklabels([-120,-90,-50,0,50,90,120],rotation=30,fontsize=15)
    #ax_hod.set_xticklabels([-120,-90,-50,0,50,90,120],rotation=30,fontsize=15)
    h = Hodograph(ax_hod, component_range=70.)#60
    h.add_grid(increment=20)
    hodo=h.plot_colormapped(u, v, wind_speed)  # Plot a line colored by wind speed
    #fig.colorbar(hodo, ax_hod,orientation='horizontal', shrink=0.74, pad=0.1)
    cbar_ax = fig.add_axes([0.14, 0.63, 0.15, 0.02])
    clevs1 =np.arange(0,30,5)
    cbar=fig.colorbar(hodo, cax=cbar_ax,orientation='horizontal', shrink=0.74, pad=0.1)
    #cbar.ax.set_xticklabels(cbar.ax.get_xticklabels(),rotation=90,fontsize=15)
    #fig.colorbar(hodo,orientation='horizontal', shrink=0.74, pad=0.1)
    # Show the plot
    plt.savefig(stapng)
    cape,cin,cape1,cin1,lfc,el,lcl,bulk=caculate(df)
    return cape,cin,cape1,cin1,lfc,el,lcl,bulk

![Diagram showing an air parcel path when raised along B-C-E compared to the surrounding air mass Temperature (T) and humidity (Tw)](https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Meteorology_thermodynamic_en.svg/330px-Meteorology_thermodynamic_en.svg.png)

In [56]:
#station=["59280","58238","58457","54511","58362","45004"]#ok 59280,58362.45004  ;no station 58238 58457 54511
#df.drop("station",axis=1,inplace=True)
df=pd.read_csv("54511_all-c.csv")

In [57]:
df

,pressure,height,temperature,dewpoint,direction,speed,u_wind,v_wind,station_number,latitude,longitude,elevation
0,1000,67.730586,25.248811,16.695721,135.118859,4.450079,-0.737789,0.274169,54511,39.93,116.28,55
1,925,729.002183,22.373581,13.061790,186.189956,10.637555,1.834095,1.699267,54511,39.93,116.28,55
2,850,1459.722707,18.017467,8.551856,200.414847,11.804585,3.727712,1.316016,54511,39.93,116.28,55
3,700,3093.034934,7.602293,-1.942904,224.656113,14.989083,5.888287,-4.112677,54511,39.93,116.28,55
4,500,5785.868852,-8.026339,-24.431038,248.540984,21.504918,14.445683,-4.664262,54511,39.93,116.28,55
5,400,7484.715536,-18.858425,-33.760503,257.314004,29.319475,22.344778,-4.714753,54511,39.93,116.28,55
6,300,9566.208791,-33.512967,-46.450879,261.021978,41.697802,33.787609,-4.802963,54511,39.93,116.28,55
7,250,10821.995590,-42.123594,-55.684675,263.506064,50.640573,41.771883,-5.248191,54511,39.93,116.28,55
8,200,12303.594270,-50.018192,-64.176655,270.011025,56.923925,48.832210,-7.857465,54511,39.93,116.28,55
9,150,14153.164280,-57.058986,-71.139376,275.794702,50.225166,43.818981,-8.937448,54511,39.93,116.28,55


In [58]:
cape0,cin0,cape,cin,cape1,cin1,lfcp,lfct,elp,elt,lclp,lclt,bulk_u,bulk_v=caculate_man(df)

In [59]:
df1=pd.DataFrame({'CAPE':[cape0],
                  'CIN':[cin0],
                  'Surface_Based_Cape':[cape],
                  'Surface_Based_Cin':[cin],
                 'Most_Unstable_CAPE':[cape1],
                  'Most_Unstable_CIN':[cin1],
                  'pressure level of free convection':[lfcp],
                  'temperature level of free convection':[lfct],
                  'pressure equilibrium level':[elp],
                  'temperature equilibrium level':[elt],
                  'pressure lifted condensation level':[lclp],
                  'temperature lifted condensation level':[lclt],
                  'bulk shear U':[bulk_u],
                  'bulk shear V':[bulk_v],
                  'stid':["54511"]
                 })

In [60]:
df1

,CAPE,CIN,Surface_Based_Cape,Surface_Based_Cin,Most_Unstable_CAPE,Most_Unstable_CIN,pressure level of free convection,temperature level of free convection,pressure equilibrium level,temperature equilibrium level,pressure lifted condensation level,temperature lifted condensation level,bulk shear U,bulk shear V,stid
0,-68055.8532761052 joule / kilogram,-41.35861512901613 joule / kilogram,-68107.968383,-101.761831,-68055.853276,-41.358615,NaN,NaN,NaN,NaN,881.978696,14.734507,3.185469,1.300914,54511


In [9]:
df1.to_csv("54511_all-caculate.csv")

out=pd.DataFrame(columns=['Surface_Based_Cape','Surface_Based_Cin','Most_Unstable_CAPE','Most_Unstable_CIN',
                  'pressure level of free convection','temperature level of free convection','pressure equilibrium level',
                  'temperature equilibrium level','pressure lifted condensation level','temperature lifted condensation level', 
                  'bulk shear U','bulk shear V','stid'])

In [67]:
#station=["59280","58238","58457","54511","58362","45004"]#ok 59280,58362.45004  ;no station 58238 58457 54511
station=["58238","58457","54511","59280","58362"]
out=pd.DataFrame(columns=["CAPE","CIN",'Surface_Based_Cape','Surface_Based_Cin','Most_Unstable_CAPE','Most_Unstable_CIN',
                  'pressure level of free convection','temperature level of free convection','pressure equilibrium level',
                  'temperature equilibrium level','pressure lifted condensation level','temperature lifted condensation level', 
                  'bulk shear U','bulk shear V','stid'])
for sta in station:
    print(sta)
    df=pd.read_csv(sta+"_rain-c.csv")
    #print(df.head)
    cape0,cin0,cape,cin,cape1,cin1,lfcp,lfct,elp,elt,lclp,lclt,bulk_u,bulk_v=caculate_man(df)
    df1=pd.DataFrame({'CAPE':[cape0],
                  'CIN':[cin0],
                  'Surface_Based_Cape':[cape],
                  'Surface_Based_Cin':[cin],
                 'Most_Unstable_CAPE':[cape1],
                  'Most_Unstable_CIN':[cin1],
                  'pressure level of free convection':[lfcp],
                  'temperature level of free convection':[lfct],
                  'pressure equilibrium level':[elp],
                  'temperature equilibrium level':[elt],
                  'pressure lifted condensation level':[lclp],
                  'temperature lifted condensation level':[lclt],
                  'bulk shear U':[bulk_u],
                  'bulk shear V':[bulk_v],
                  'stid':[sta]
                 })
    print(df1)
    out=out.append(df1)

58238
                                   CAPE                   CIN  \
0  -2696.9801010859505 joule / kilogram  0.0 joule / kilogram   

   Surface_Based_Cape  Surface_Based_Cin  Most_Unstable_CAPE  \
0        -2722.717052         -11.767311        -2392.271441   

   Most_Unstable_CIN  pressure level of free convection  \
0                0.0                                NaN   

   temperature level of free convection  pressure equilibrium level  \
0                                   NaN                         NaN   

   temperature equilibrium level  pressure lifted condensation level  \
0                            NaN                          962.157584   

   temperature lifted condensation level  bulk shear U  bulk shear V   stid  
0                               20.15582      2.183104      0.950628  58238  
58457
                                   CAPE                   CIN  \
0  -1376.6212593883333 joule / kilogram  0.0 joule / kilogram   

   Surface_Based_Cape  Surface_Bas

In [68]:
out.to_csv("rain.csv")

In [32]:
out

,Surface_Based_Cape,Surface_Based_Cin,Most_Unstable_CAPE,Most_Unstable_CIN,pressure level of free convection,temperature level of free convection,pressure equilibrium level,temperature equilibrium level,pressure lifted condensation level,temperature lifted condensation level,bulk shear U,bulk shear V,stid
0,0.400587,-221.757854,0.400587,-217.562711,512.306408,-2.949822,488.009963,-5.044092,933.170165,20.335528,2.770414,2.447238,58238
0,-1230.196188,-37.342834,-1168.206189,0.000000,NaN,NaN,NaN,NaN,929.397041,20.247247,2.204493,1.996955,58457
0,-68129.666632,-101.761831,-68077.551526,-41.358615,NaN,NaN,NaN,NaN,881.978696,14.734507,3.185469,1.300914,54511


In [27]:
station=["58238","58457","54511"]
out=pd.DataFrame(columns=['Surface_Based_Cape','Surface_Based_Cin','Most_Unstable_CAPE','Most_Unstable_CIN',
                  'pressure level of free convection','temperature level of free convection','pressure equilibrium level',
                  'temperature equilibrium level','pressure lifted condensation level','temperature lifted condensation level', 
                  'bulk shear U','bulk shear V','stid'])
for sta in station:
    print(sta)
    df=pd.read_csv(sta+"_rain.csv")
    #print(df.head)
    cape,cin,cape1,cin1,lfcp,lfct,elp,elt,lclp,lclt,bulk_u,bulk_v=caculate_man(df)
    df1=pd.DataFrame({'Surface_Based_Cape':[cape],
                  'Surface_Based_Cin':[cin],
                 'Most_Unstable_CAPE':[cape1],
                  'Most_Unstable_CIN':[cin1],
                  'pressure level of free convection':[lfcp],
                  'temperature level of free convection':[lfct],
                  'pressure equilibrium level':[elp],
                  'temperature equilibrium level':[elt],
                  'pressure lifted condensation level':[lclp],
                  'temperature lifted condensation level':[lclt],
                  'bulk shear U':[bulk_u],
                  'bulk shear V':[bulk_v],
                  'stid':[sta]
                 })
    print(df1)
    out=out.append(df1)

58238
   Surface_Based_Cape  Surface_Based_Cin  Most_Unstable_CAPE  \
0        -2722.717058         -11.767311        -2392.271443   

   Most_Unstable_CIN  pressure level of free convection  \
0                0.0                                NaN   

   temperature level of free convection  pressure equilibrium level  \
0                                   NaN                         NaN   

   temperature equilibrium level  pressure lifted condensation level  \
0                            NaN                          962.157584   

   temperature lifted condensation level  bulk shear U  bulk shear V   stid  
0                               20.15582      2.183104      0.950628  58238  
58457
   Surface_Based_Cape  Surface_Based_Cin  Most_Unstable_CAPE  \
0        -1404.633823         -10.543266         -977.058628   

   Most_Unstable_CIN  pressure level of free convection  \
0                0.0                                NaN   

   temperature level of free convection  pressur

In [28]:
out.to_csv("rain.csv")

In [29]:
out

,Surface_Based_Cape,Surface_Based_Cin,Most_Unstable_CAPE,Most_Unstable_CIN,pressure level of free convection,temperature level of free convection,pressure equilibrium level,temperature equilibrium level,pressure lifted condensation level,temperature lifted condensation level,bulk shear U,bulk shear V,stid
0,-2722.717058,-11.767311,-2392.271443,0.00000,NaN,NaN,NaN,NaN,962.157584,20.155820,2.183104,0.950628,58238
0,-1404.633823,-10.543266,-977.058628,0.00000,NaN,NaN,NaN,NaN,965.265863,21.646918,5.583547,3.515966,58457
0,-2982.083189,-96.207956,10.308191,-214.33747,NaN,NaN,NaN,NaN,896.360963,16.103421,2.288505,5.222878,54511


In [14]:
station=["58362"]
out=pd.DataFrame(columns=['Surface_Based_Cape','Surface_Based_Cin','Most_Unstable_CAPE','Most_Unstable_CIN',
                  'pressure level of free convection','temperature level of free convection','pressure equilibrium level',
                  'temperature equilibrium level','pressure lifted condensation level','temperature lifted condensation level', 
                  'bulk shear U','bulk shear V','stid'])
for sta in station:
    print(sta)
    df=pd.read_csv(sta+"_all.csv")
    #print(df.head)
    cape,cin,cape1,cin1,lfcp,lfct,elp,elt,lclp,lclt,bulk_u,bulk_v=caculate_man(df)
    df1=pd.DataFrame({'Surface_Based_Cape':[cape],
                  'Surface_Based_Cin':[cin],
                 'Most_Unstable_CAPE':[cape1],
                  'Most_Unstable_CIN':[cin1],
                  'pressure level of free convection':[lfcp],
                  'temperature level of free convection':[lfct],
                  'pressure equilibrium level':[elp],
                  'temperature equilibrium level':[elt],
                  'pressure lifted condensation level':[lclp],
                  'temperature lifted condensation level':[lclt],
                  'bulk shear U':[bulk_u],
                  'bulk shear V':[bulk_v],
                  'stid':[sta]
                 })
    print(df1)
    out=out.append(df1)

58362


IndexError: index 0 is out of bounds for axis 0 with size 0

In [25]:
df=pd.read_csv("58238_rain.csv")

In [26]:
cape,cin,cape1,cin1,lfcp,lfct,elp,elt,lclp,lclt,bulk_u,bulk_v=caculate_man(df)

In [17]:
df1=pd.read_csv("58238_all.csv")

In [18]:
df1.head()

,pressure,height,temperature,dewpoint,direction,speed,u_wind,v_wind,station_number,latitude,longitude,elevation
0,1000.0,58.718056,26.191389,21.459306,134.854167,5.370833,-2.535938,0.617901,58238.0,32.0,118.8,7.0
1,925.0,739.155531,23.016429,17.836364,166.998905,13.509310,-0.760613,3.247403,58238.0,32.0,118.8,7.0
2,850.0,1473.169770,19.136911,12.574918,184.830230,13.828039,2.310382,2.684908,58238.0,32.0,118.8,7.0
3,700.0,3119.366922,10.766046,-0.054326,216.451260,15.069003,7.440577,2.009880,58238.0,32.0,118.8,7.0
4,500.0,5851.831140,-3.849123,-18.344189,238.196272,21.286184,15.719159,2.211456,58238.0,32.0,118.8,7.0


In [19]:
df2=df1

In [20]:
df2

,pressure,height,temperature,dewpoint,direction,speed,u_wind,v_wind,station_number,latitude,longitude,elevation
0,1000.0,58.718056,26.191389,21.459306,134.854167,5.370833,-2.535938,0.617901,58238.0,32.0,118.8,7.0
1,925.0,739.155531,23.016429,17.836364,166.998905,13.509310,-0.760613,3.247403,58238.0,32.0,118.8,7.0
2,850.0,1473.169770,19.136911,12.574918,184.830230,13.828039,2.310382,2.684908,58238.0,32.0,118.8,7.0
3,700.0,3119.366922,10.766046,-0.054326,216.451260,15.069003,7.440577,2.009880,58238.0,32.0,118.8,7.0
4,500.0,5851.831140,-3.849123,-18.344189,238.196272,21.286184,15.719159,2.211456,58238.0,32.0,118.8,7.0
5,400.0,7580.944018,-13.815477,-27.137212,237.616905,26.633370,20.607689,1.815327,58238.0,32.0,118.8,7.0
6,300.0,9708.309550,-28.108123,-40.712184,233.896817,34.302964,25.352466,1.406471,58238.0,32.0,118.8,7.0
7,250.0,10990.592755,-37.940834,-50.493743,228.962678,39.401756,27.612505,-0.182628,58238.0,32.0,118.8,7.0
8,200.0,12487.691548,-49.775960,-62.156044,228.057080,43.970362,29.008824,-3.479039,58238.0,32.0,118.8,7.0
9,150.0,14314.039517,-62.631504,-74.543373,225.104281,40.379802,24.919284,-7.551198,58238.0,32.0,118.8,7.0


---

def caculate_all(df):
    df=df.dropna()
    p = df['pressure'].values * units(profile_units['pressure'])
    #p = df.index.values * units(profile_units['pressure']) 
    T = df['temperature'].values * units(profile_units['temperature'])
    Td = df['dewpoint'].values * units(profile_units['dewpoint'])
    u = df['u_wind'].values * units(profile_units['u_wind'])
    v = df['v_wind'].values * units(profile_units['v_wind'])
    wind_speed = df['speed'].values * units(profile_units['speed'])
    cape,cin=mpcalc.surface_based_cape_cin(p, T, Td)
    cape1,cin1=mpcalc.most_unstable_cape_cin(p, T, Td)
    lfcp,lfct=mpcalc.lfc(p, T, Td)
    el=mpcalc.el(p,T,Td)
    lcl=mpcalc.lcl(p[0],T[0],Td[0])
    bulk_u,bulk_v=mpcalc.bulk_shear(p,u,v)
    return cape,cin,cape,cin,lfcp,lfct,el,lcl,bulk_u,bulk_v

def caculate(df):
    df=df.dropna()
    p = df['pressure'].values * units(profile_units['pressure'])
    #p = df.index.values * units(profile_units['pressure']) 
    T = df['temperature'].values * units(profile_units['temperature'])
    Td = df['dewpoint'].values * units(profile_units['dewpoint'])
    u = df['u_wind'].values * units(profile_units['u_wind'])
    v = df['v_wind'].values * units(profile_units['v_wind'])
    wind_speed = df['speed'].values * units(profile_units['speed'])
    cape,cin=mpcalc.surface_based_cape_cin(p, T, Td)
    cape1,cin1=mpcalc.most_unstable_cape_cin(p, T, Td)
    cape_man=cape.magnitude
    cape_man1=cape1.magnitude
    cin_man=cin.magnitude
    cin_man1=cin1.magnitude  
    lfc=mpcalc.lfc(p, T, Td)
    lfc_man=[]
    for i in range(len(lfc)):
        a=lfc[i].magnitude
        lfc_man=np.append(lfc_man,a)
    el=mpcalc.el(p,T,Td)
    el_man=[]
    for i in range(len(el)):
        a=el[i].magnitude
        lcl_man=np.append(el_man,a)
    lcl=mpcalc.lcl(p,T,Td)
    lcl_man=[]
    for i in range(len(lcl)):
        a=lcl[i].magnitude
        lcl_man=np.append(lcl_man,a)
    bulk=mpcalc.bulk_shear(p,u,v)
    bulk_man=[]
    for i in range(len(bulk)):
        a=bulk[i].magnitude
        bulk_man=np.append(bulk_man,a)
    return cape_man,cin_man,cape_man1,cin_man1,lfc_man,el_man,lcl_man,bulk_man